# [オンライン開催]PyTorchで学ぶ深層学習入門第8回

## Section0. 前段
この勉強会では、グラフニューラルネットワークについて説明します。  
※参考
[**ネットワーク理論最前線- 基礎から応用まで -** 産業技術総合研究所(小島氏)](https://staff.aist.go.jp/k.kojima/archives/smips2006.pdf)  
  
グラフ ニューラル ネットワーク (GNN) は、ソーシャルネットワーク、ナレッジグラフ、レコメンドシステム(推薦システム)、バイオインフォマティクスなどの分野を含むアプリケーションと研究の両方で活躍しているネットワークとなります。 GNN の背後にある理論と数学は最初は複雑に見えるかもしれませんが、これらのモデルの実装は非常に簡単で、仕組みを理解するのに役立ちます。したがって、GNN の基本的なネットワーク層である、グラフ畳み込みとAttention層の実装について説明します。最後に、ノードレベル、エッジレベル、およびグラフレベルのタスクに GNN を適用します。

いつも通り、標準ライブラリをインポートすることから始めます。既に扱った、PyTorch Lightning を使用します。

In [1]:
## Standard libraries
import os
import json
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/vol7"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

<ipython-input-1-bb5212b8e35a>:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export
INFO:lightning_fabric.utilities.seed:Global seed set to 42


cuda:0


すでに訓練された、学習されたモデルがあるのでそれら必要なリソースをダウンロードします

In [2]:
import urllib.request
from urllib.error import HTTPError
# Github URL where saved models are stored for this tutorial
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# Files to download
pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# Create checkpoint path if it doesn't exist yet
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# For each file, check whether it already exists. If not, try downloading it.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Downloading {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

## Section1. Graph Neural Networks (グラフニューラルネットワーク)

### Section1-1. Graph representation

グラフに対する特定のニューラル ネットワーク操作の議論を始める前に、グラフを表現する方法を検討する必要があります。  
数学的には、グラフ $\mathcal{G}$ はノード/頂点のセット $V$ とエッジ/リンクのセット $E$ のタプルとして定義されます: $\mathcal{G}=(V,E) $.各エッジは 2 つの頂点のペアであり、それらの間の接続を表します。たとえば、次のグラフを見てみましょう。

![グラフ](https://drive.google.com/uc?id=1hPJyE-a-HUWePY6Ttxl_4G9hx6OvYQzE)

頂点は $V=\{1,2,3,4\}$ で、エッジ $E=\{(1,2), (2,3), (2,4), (3,4)\}$.  
簡単にするために、グラフは無向(つまりどこに向かうかについては定義しない)であると想定しているため、$(2,1)$ のようなミラーリングされたペアを追加しないことに注意してください。アプリケーションでは、多くの場合、頂点とエッジに特定の属性を持たせることができ、エッジを方向付けることさえできます。問題は、行列演算の効率的な方法でこの多様性をどのように表現できるのか、です。通常、エッジについては、隣接行列または対になった頂点インデックスのリストの 2 つのバリアントのどちらかを決定します。

 [**隣接行列**](https://mathwords.net/gurahu)  $A$ は正方行列で、その要素は頂点のペアが隣接しているかどうか、つまり接続されているかどうかを示します。最も単純なケースでは、ノード $i$ から $j$ への接続がある場合、$A_{ij}$ は 1 であり、それ以外の場合は 0 です。グラフにエッジ属性または異なるカテゴリのエッジがある場合、この情報は次のようになります。マトリックスにも追加。無向グラフの場合、$A$ は対称行列 ($A_{ij}=A_{ji}$) であることに注意してください。上記のグラフの例では、次の隣接行列があります。

$$
A = \begin{bmatrix}
    0 & 1 & 0 & 0\\
    1 & 0 & 1 & 1\\
    0 & 1 & 0 & 1\\
    0 & 1 & 1 & 0
\end{bmatrix}
$$

グラフをエッジのリストとして表現すると、メモリと (場合によっては) 計算の点でより効率的になりますが、隣接行列を使用すると、より直感的で簡単に実装できます。以下の実装では、コードを単純にするために隣接行列に依存します。ただし、一般的なライブラリはエッジ リストを使用します。これについては後で詳しく説明します。
別の方法として、エッジのリストを使用して疎な隣接行列を定義することもできます。これにより、あたかも密な行列であるかのように作業できますが、よりメモリ効率の高い操作が可能になります。 PyTorch はサブパッケージ `torch.sparse` ([ドキュメント](https://pytorch.org/docs/stable/sparse.html)) でこれをサポートしますが、これはまだベータ段階です (API は将来変更される可能性があります) ）。

※Wikipediaより [隣接行列](https://ja.wikipedia.org/wiki/%E9%9A%A3%E6%8E%A5%E8%A1%8C%E5%88%97)  
![任意の画像名を付ける](https://drive.google.com/uc?id=15nKtAm1O2Dp-tPvm7DxhtUu6l5CfIpjv)

![任意の画像名を付ける](https://drive.google.com/uc?id=1KdDzfBvMkcx0nS0tIxvCqLChJJGPrC1H)

![任意の画像名を付ける](https://drive.google.com/uc?id=1YsXhkmhCocGPOTNhy8qB9KQREN9adJhD)


### Section1-2. Graph Convolutions

グラフ畳み込みネットワークは、2016 年にアムステルダム大学で [Kipf ら](https://openreview.net/pdf?id=SJU4ayYgl) によって導入されました。彼らはまた、このトピックについて素晴らしい [ブログ](https://tkipf.github.io/graph-convolutional-networks/) を書いています。GCN について別の視点から読みたい場合は、このトピックをお勧めします。 GCN は、「フィルター」パラメーターが通常、グラフ内のすべての場所で共有されるという意味で、画像の畳み込み(「画像の各ピクセルに対して、近隣のピクセル (3×3 あるいは 5×5 など) に特定の係数をかけて合計する」処理)に似ています。  
同時に、GCN はメッセージ パッシング メソッドに依存しています。これは、頂点が隣接する頂点と情報を交換し、互いに「メッセージ」を送信することを意味します。数式を見る前に、GCN がどのように機能するかを視覚的に理解してみましょう。最初のステップは、各ノードがすべての近隣ノードに送信したいメッセージを表す特徴ベクトルを作成することです。 2 番目のステップでは、ノードが隣接ノードごとに 1 つのメッセージを受信するように、メッセージがネイバー(隣接するもの)に送信されます。以下に、サンプル グラフの 2 つのステップを視覚化しました。

![任意の画像名を付ける](https://drive.google.com/uc?id=1hlA7wbpPVVkeAi0jQxwAGPVIZ3iO5Sl3)

これをより数学的な用語で定式化したい場合は、ノードが受信するすべてのメッセージを結合する方法を最初に決定する必要があります。メッセージの数はノードによって異なるため、任意の数に対して機能する操作が必要です。したがって、通常の方法は、合計または平均を取ることです。ノード $H^{(l)}$ の以前の機能を考えると、GCN レイヤーは次のように定義されます。

$$H^{(l+1)} = \sigma\left(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)}\right)$$

$W^{(l)}$ は、入力フィーチャをメッセージに変換する重みパラメータです ($H^{(l)}W^{(l)}$)。隣接行列 $A$ に恒等行列を追加して、各ノードがそれ自体にも独自のメッセージを送信するようにします: $\hat{A}=A+I$.最後に、合計する代わりに平均を取るために、行列 $\hat{D}$ を計算します。これは $D_{ii}$ がノード $i$ の隣接ノードの数を表す対角行列です。 $\sigma$ は任意の活性化関数を表し、必ずしもシグモイドとは限りません (通常、GNN では ReLU ベースの活性化関数が使用されます)。

PyTorch で GCN レイヤーを実装すると、テンソルの柔軟な操作を利用できます。行列 $\hat{D}$ を定義する代わりに、合計されたメッセージを後で近傍の数で単純に割ることができます。さらに、重みマトリックスを線形レイヤーに置き換えます。これにより、さらにバイアスを追加できます。 PyTorch モジュールとして記述された GCN レイヤーは、次のように定義されます。

In [3]:
class GCNLayer(nn.Module):
    
    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections. 
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats

GCN レイヤーをさらに理解するために、上の例のグラフに適用できます。まず、いくつかのノード機能と、自己接続が追加された隣接行列を指定しましょう。

In [4]:
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


次にGCNレイヤーを適用しましょう。簡単にするために、入力特徴がメッセージと等しくなるように、線形重み行列を単位行列として初期化します。これにより、メッセージ パッシング操作の検証が容易になります。

In [5]:
layer = GCNLayer(c_in=2, c_out=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


上記のとおり、最初のノードの出力値は、それ自体と 2 番目のノードの平均です。同様に、他のすべてのノードを確認できます。ただし、GNN では、隣接ノードを超えたノード間の機能交換も許可する必要があります。これは、複数の GCN レイヤーを適用することで実現できます。これにより、GNN の最終的なレイアウトが得られます。 GNN は、一連の GCN レイヤーと ReLU などの非線形性によって構築できます。視覚化については、以下を参照してください (図の引用元 - [Thomas Kipf, 2016](https://tkipf.github.io/graph-convolutional-networks/))。

![任意の画像名を付ける](https://drive.google.com/uc?id=1AZCLKPlXJt-p7reJwMKOUlMhiwBFgGvz)

ただし、上記の例を見るとわかる 1 つの問題は、ノード 3 と 4 の出力フィーチャが同じであることです。これは、同じ隣接ノード (それ自体を含む) があるためです。したがって、すべてのメッセージを平均するだけで、GCN 層によってネットワークにノード固有の情報を忘れてしまうことになります。これらについては、複数の改善が提案されています。最も単純なオプションは残差接続を使用することかもしれませんが、より一般的なアプローチは、自己接続の重みを高くするか、自己接続に別の重み行列を定義することです。または、以前紹介の概念であるアテンションを再検討することもできます。

### Graph Attention 

第5回を参加された方は、入力クエリと要素キーについて学んだかと思います。入力クエリと要素のキーに基づいて動的に計算された重みを使用して、複数の要素の加重平均について説明することに注意してください ([第5回のノートブック](https://github.com/kazuhiroSato2022/learning_deepL_pytorch/blob/main/vol5/learning_deepL_pytorch_vol5.ipynb) )。この概念はグラフにも同様に適用できます。その 1 つが Graph Attention Network (GAT と呼ばれ、[Velickovic et al., 2017](https://arxiv.org/abs/1710.10903) によって提案されました) です。 GCN と同様に、グラフ アテンション レイヤーは、線形レイヤー/重み行列を使用して各ノードのメッセージを作成します。注意メカニズム部分については、ノード自体からのメッセージをクエリとして使用し、キーと値の両方として平均化するメッセージを使用します (これには自身へのメッセージも含まれることに注意してください)。スコア関数 $f_{attn}$ は、クエリとキーを単一の値にマップする 1 層 MLP として実装されます。 MLP は次のようになります (図のクレジット - [Velickovic et al.](https://arxiv.org/abs/1710.10903)):

![任意の画像名を付ける](https://drive.google.com/uc?id=19vr-hFmCGUCDlmbSyAtVHPSpu8_5Nxv-)

$h_i$ と $h_j$ は、それぞれノード $i$ と $j$ からの元の特徴であり、重み行列として $\mathbf{W}$ を使用して層のメッセージを表します。 $\mathbf{a}$ は MLP の重み行列で、形状は $[1,2\times d_{\text{message}}]$ で、$\alpha_{ij}$ はノード $i$ から $j$。計算は次のように記述できます。

$$\alpha_{ij} = \frac{\exp\left(\text{LeakyReLU}\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_j\right]\right)\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\text{LeakyReLU}\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_k\right]\right)\right)}$$

演算子 $||$ は連結を表し、$\mathcal{N}_i$ はノード $i$ の隣接ノードのインデックスです。通常のプラクティスとは対照的に、要素に対するソフトマックスの前に非線形性 (ここでは LeakyReLU) を適用することに注意してください。最初は一般的でない設計変更のように見えますが、アテンションが元の入力に依存することが重要です。具体的には、非線形性を少し取り除いて、式を単純化してみましょう。

$$
\begin{split}
    \alpha_{ij} & = \frac{\exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_j\right]\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_k\right]\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\
\end{split}
$$

非線形性がなければ、$h_i$ を使用した注意項は実際にそれ自体を打ち消し、注意がノード自体から独立していることがわかります。したがって、同じ隣接ノードを持つノードに対して同じ出力フィーチャを作成するという GCN と同じ問題が発生します。ここの問題に、LeakyReLU が重要であり、$h_i$ への依存関係を追加する理由です。

すべてのアテンション因子を取得したら、加重平均を実行して各ノードの出力特徴を計算できます。

$$h_i'=\sigma\left(\sum_{j\in\mathcal{N}_i}\alpha_{ij}\mathbf{W}h_j\right)$$

$\sigma$ は、GCN レイヤーのように、さらに別の非線形性です。視覚的には、次のようにアテンション レイヤーで渡される完全なメッセージを表すことができます (図のクレジット - [Velickovic et al.](https://arxiv.org/abs/1710.10903)):

![任意の画像名を付ける](https://drive.google.com/uc?id=1SmcisALVz-Vg-AWfCaCpGhJ2AMbOaUH1)

Graph Attention ネットワークの表現力を高めるために、[Velickovic et al.](https://arxiv.org/abs/1710.10903) は、Transformers の Multi-Head Attention ブロックと同様に、グラフ アテンション ネットワークを複数のヘッドに拡張することを提案しました。これにより、$N$ 個のアテンション レイヤーが並行して適用されます。上の画像では、後で連結される 3 つの異なる色の矢印 (緑、青、紫) として視覚化されています。平均は、ネットワークの最後の予測レイヤーにのみ適用されます。

グラフアテンションレイヤーについて詳しく説明した後、以下で実装できます。

In [6]:
class GATLayer(nn.Module):
    
    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensionality of input features
            c_out - Dimensionality of output features
            num_heads - Number of heads, i.e. attention mechanisms to apply in parallel. The 
                        output features are equally split up over the heads if concat_heads=True.
            concat_heads - If True, the output of the different heads is concatenated instead of averaged.
            alpha - Negative slope of the LeakyReLU activation.
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Number of output features must be a multiple of the count of heads."
            c_out = c_out // num_heads
        
        # Sub-modules and parameters needed in the layer
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out)) # One per head
        self.leakyrelu = nn.LeakyReLU(alpha)
        
        # Initialization from the original implementation
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)
        
    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Input features of the node. Shape: [batch_size, c_in]
            adj_matrix - Adjacency matrix including self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - If True, the attention weights are printed during the forward pass (for debugging purposes)
        """
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)
        
        # Apply linear layer and sort nodes by head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)
        
        # We need to calculate the attention logits for every edge in the adjacency matrix 
        # Doing this on all possible combinations of nodes is very expensive
        # => Create a tensor of [W*h_i||W*h_j] with i and j being the indices of all edges
        edges = adj_matrix.nonzero(as_tuple=False) # Returns indices where the adjacency matrix is not 0 => edges
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # Index select returns a tensor with node_feats_flat being indexed at the desired positions along dim=0
        
        # Calculate attention MLP output (independent for each head)
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a) 
        attn_logits = self.leakyrelu(attn_logits)
        
        # Map list of attention values back into a matrix
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)
        
        # Weighted average of attention
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", attn_probs.permute(0, 3, 1, 2))
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)
        
        # If heads should be concatenated, we can do this by reshaping. Otherwise, take mean
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)
        
        return node_feats 

繰り返しになりますが、動作原理をよりよく理解するために、上記のサンプル グラフにグラフ アテンション レイヤーを適用します。前と同様に、入力層は単位行列として初期化されますが、 $\mathbf{a}$ を任意の数値のベクトルに設定して、異なるアテンション値を取得します。 2 つの頭を使用して、レイヤー内で動作する並列の独立した注意メカニズムを示します。

In [7]:
layer = GATLayer(2, 2, num_heads=2)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Attention probs
 tensor([[[[0.3543, 0.6457, 0.0000, 0.0000],
          [0.1096, 0.1450, 0.2642, 0.4813],
          [0.0000, 0.1858, 0.2885, 0.5257],
          [0.0000, 0.2391, 0.2696, 0.4913]],

         [[0.5100, 0.4900, 0.0000, 0.0000],
          [0.2975, 0.2436, 0.2340, 0.2249],
          [0.0000, 0.3838, 0.3142, 0.3019],
          [0.0000, 0.4018, 0.3289, 0.2693]]]])
Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1.2913, 1.9800],
         [4.2344, 3.7725],
         [4.6798, 4.8362],
         [4.5043, 4.7351]]])


少なくとも 1 つのヘッドと 1 つのノードについて、アテンション マトリックスを自分で計算することをお勧めします。 $i$ と $j$ の間にエッジが存在しない場合、エントリは 0 です。他のものについては、さまざまなアテンション確率のセットが見られます。さらに、ノード 3 と 4 の出力フィーチャは、同じ隣接フィーチャを持っていますが、情報は異なっています。

## Section2. PyTorch Geometric

隣接行列を使用したグラフ ネットワークの実装は単純で簡単ですが、大規模なグラフでは計算コストが高くなる可能性があることを前に述べました。実際のグラフの多くは 20 万ノードを超える可能性があり、隣接行列ベースの実装では失敗します。 GNN を実装する際には多くの最適化が可能であり、幸いなことに、そのようなレイヤーを提供するパッケージが存在します。 PyTorch の最も一般的なパッケージは、[PyTorch ジオメトリック](https://pytorch-geometric.readthedocs.io/en/latest/) と [ディープ グラフ ライブラリ](https://www.dgl.ai/) (後者は実際にはフレームワークに依存しません)。どちらを使用するかは、計画しているプロジェクトと個人的な好みによって異なります。この勉強会では、PyTorch の一部として PyTorch Geometricを見ていきます。 PyTorch Lightning と同様に、PyTorch Geometric はデフォルトでは GoogleColab にインストールされていません (実際には、実際には不要な多くの依存関係があるため、`dl2021` 環境にもインストールされていません)。したがって、以下でインポートおよび/またはインストールしましょう。

In [8]:
# torch geometric
try: 
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version. 
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details 
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric 
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 77.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 88.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 885.2/885.2 KB 48.7 M

PyTorch ジオメトリックは、上で実装した GCN および GAT レイヤーを含む一連の一般的なグラフ レイヤーを提供します。さらに、PyTorch の torchvision と同様に、トレーニングを簡素化するために、共通のグラフ データセットとそれらの変換を提供します。上記の実装と比較して、PyTorch ジオメトリックはインデックス ペアのリストを使用してエッジを表します。このライブラリの詳細は、実験でさらに詳しく調べます。

以下のタスクでは、多数のグラフ レイヤーから選択できるようにしたいと考えています。したがって、文字列を使用して辞書にアクセスするために、辞書の下に再度定義します。

In [9]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

GCN と GAT に加えて、レイヤー `geom_nn.GraphConv` を追加しました ([ドキュメント](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GraphConv) ）。 GraphConv は、自己接続用の個別の重み行列を持つ GCN です。数学的には、これは次のようになります。

$$
\mathbf{x}_i^{(l+1)} = \mathbf{W}^{(l + 1)}_1 \mathbf{x}_i^{(l)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{j \in \mathcal{N}_i} \mathbf{x}_j^{(l)}
$$

この式では、近隣のメッセージが平均化される代わりに追加されます。ただし、PyTorch ジオメトリックは、合計、平均化、および最大プーリングを切り替える引数 `aggr` を提供します。

## Section3. グラフ構造の実験

グラフ構造のデータに対するタスクは、ノード レベル、エッジ レベル、およびグラフ レベルの 3 つのグループにグループ化できます。さまざまなレベルは、分類/回帰を実行するレベルを表します。以下では、3 つのタイプすべてについて詳しく説明します。  

参考ブログ  
* [PyTorch GeometricでGraph Neural Network（GNN）入門](https://cpp-learning.com/pytorch-geometric/)
* [Understanding Convolutions on Graphs](https://distill.pub/2021/understanding-gnns/)

### Section3-1. ノードレベルのタスク: 半教師付きノード分類

ノードレベルのタスクには、グラフ内のノードを分類するという目標があります。通常、1000 個を超えるノードを含む単一の大きなグラフが与えられ、そのうちの一定量のノードにラベルが付けられます。トレーニング中にこれらのラベル付けされた例を分類することを学び、ラベル付けされていないノードに一般化しようとします。

この勉強会で使用するオープンデータセットは、論文間の引用ネットワークである Cora データセット（[【グラフ構造】論文の引用データセットCoraを利用する](https://disassemble-channel.com/graph-cora-datasets/)）です。 Cora は 2,708 の科学出版物で構成されており、論文間の引用を表す相互リンクがあります。タスクは、各出版物を 7 つのクラスのいずれかに分類することです。各出版物は、bag-of-words ベクトルで表されます。これは、出版物ごとに 1433 要素のベクトルがあることを意味します。特徴 $i$ の 1 は、事前定義された辞書の $i$ 番目の単語が記事にあることを示します。バイナリとしてのBag of Word表現は、非常に単純なエンコーディングが必要な場合に一般的に使用され、ネットワークで期待される単語の直感が既にある場合に使用されます。はるかに優れたアプローチが存在しますが、これについては NLPなど自然言語処理としての別勉強会の枠で紹介します。

以下のデータセットをロードします。

In [10]:
cora_dataset = torch_geometric.datasets.Planetoid(root=DATASET_PATH, name="Cora")

Processing...
Done!


PyTorch ジオメトリックがグラフ データをどのように表すかを見てみましょう。グラフは 1 つですが、PyTorch ジオメトリックは他のデータセットとの互換性のためにデータセットを返すことに注意してください。

In [11]:
cora_dataset[0]

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

グラフは、`Data` オブジェクト ([ドキュメント](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data)) によって表され、標準の Python 名前空間。エッジ インデックス テンソルは、グラフ内のエッジのリストであり、無向グラフの各エッジのミラー バージョンを含みます。 「train_mask」、「val_mask」、および「test_mask」は、トレーニング、検証、およびテストに使用するノードを示すブール値のマスクです。 「x」テンソルは 2708 出版物の特徴テンソルで、「y」はすべてのノードのラベルです。

データを確認したら、単純なグラフ ニューラル ネットワークを実装できます。 GNN は、一連のグラフ レイヤー (GCN、GAT、または GraphConv)、活性化関数としての ReLU、正則化のためのドロップアウトを適用します。具体的な実装については、以下を参照してください。

In [12]:
class GNNModel(nn.Module):
    
    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of "hidden" graph layers
            layer_name - String of the graph layer to use
            dp_rate - Dropout rate to apply throughout the network
            kwargs - Additional arguments for the graph layer (e.g. number of heads for GAT)
        """
        super().__init__()
        gnn_layer = gnn_layer_by_name[layer_name]
        
        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=in_channels, 
                          out_channels=out_channels,
                          **kwargs),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden
        layers += [gnn_layer(in_channels=in_channels, 
                             out_channels=c_out,
                             **kwargs)]
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x, edge_index):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
        """
        for l in self.layers:
            # For graph layers, we need to add the "edge_index" tensor as additional input
            # All PyTorch Geometric graph layer inherit the class "MessagePassing", hence
            # we can simply check the class type.
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                x = l(x)
        return x

ノード レベルのタスクでは、各ノードに個別に適用される MLP ベースラインを作成することをお勧めします。このようにして、グラフ情報をモデルに追加することで実際に予測が改善されるかどうかを検証できます。また、ノードごとの機能が、特定のクラスを明確に指し示すのに十分な表現力を備えている可能性もあります。これを確認するために、以下の単純な MLP を実装します。

In [13]:
class MLPModel(nn.Module):
    
    def __init__(self, c_in, c_hidden, c_out, num_layers=2, dp_rate=0.1):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of the output features. Usually number of classes in classification
            num_layers - Number of hidden layers
            dp_rate - Dropout rate to apply throughout the network
        """
        super().__init__()
        layers = []
        in_channels, out_channels = c_in, c_hidden
        for l_idx in range(num_layers-1):
            layers += [
                nn.Linear(in_channels, out_channels),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden
        layers += [nn.Linear(in_channels, c_out)]
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x, *args, **kwargs):
        """
        Inputs:
            x - Input features per node
        """
        return self.layers(x)

最後に、トレーニング、検証、およびテストを処理する PyTorch Lightning モジュールにモデルをマージできます。

In [14]:
class NodeLevelGNN(pl.LightningModule):
    
    def __init__(self, model_name, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()
        
        if model_name == "MLP":
            self.model = MLPModel(**model_kwargs)
        else:
            self.model = GNNModel(**model_kwargs)
        self.loss_module = nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index
        x = self.model(x, edge_index)
        
        # Only calculate the loss on the nodes corresponding to the mask
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, f"Unknown forward mode: {mode}"
        
        loss = self.loss_module(x[mask], data.y[mask])
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return loss, acc

    def configure_optimizers(self):
        # We use SGD here, but Adam works as well 
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

Lightning モジュールに加えて、以下のトレーニング関数を定義します。単一のグラフがあるため、データ ローダーにバッチ サイズ 1 を使用し、トレーニング、検証、およびテスト セットに同じデータ ローダーを共有します (マスクは Lightning モジュール内で選択されます)。さらに、引数 enable_progress_bar を False に設定します。これは通常、エポックごとの進行状況を示しますが、エポックは 1 つのステップのみで構成されます。コードの残りの部分は、第5回ですでに説明したと類似したコードとなってます。

In [15]:
def train_node_classifier(model_name, dataset, **model_kwargs):
    pl.seed_everything(42)
    node_data_loader = geom_data.DataLoader(dataset, batch_size=1)
    
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "NodeLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=200,
                         enable_progress_bar=False) # False because epoch size is 1
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"NodeLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = NodeLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything()
        model = NodeLevelGNN(model_name=model_name, c_in=dataset.num_node_features, c_out=dataset.num_classes, **model_kwargs)
        trainer.fit(model, node_data_loader, node_data_loader)
        model = NodeLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    
    # Test best model on the test set
    test_result = trainer.test(model, node_data_loader, verbose=False)
    batch = next(iter(node_data_loader))
    batch = batch.to(model.device)
    _, train_acc = model.forward(batch, mode="train")
    _, val_acc = model.forward(batch, mode="val")
    result = {"train": train_acc,
              "val": val_acc,
              "test": test_result[0]['test_acc']}
    return model, result

最後に、モデルをトレーニングできます。まず、単純な MLP をトレーニングしましょう。

In [16]:
# Small function for printing the test scores
def print_results(result_dict):
    if "train" in result_dict:
        print(f"Train accuracy: {(100.0*result_dict['train']):4.2f}%")
    if "val" in result_dict:
        print(f"Val accuracy:   {(100.0*result_dict['val']):4.2f}%")
    print(f"Test accuracy:  {(100.0*result_dict['test']):4.2f}%")

In [17]:
node_mlp_model, node_mlp_result = train_node_classifier(model_name="MLP",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)

print_results(node_mlp_result)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.2 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../saved_models/tutorial7/NodeLevelMLP.ckpt`


Found pretrained model, loading...


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Train accuracy: 95.00%
Val accuracy:   49.80%
Test accuracy:  60.60%


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2708. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


MLP は、高次元の入力特徴のためにトレーニング データセットにオーバーフィットする可能性がありますが、テスト セットではあまりうまく機能しません。グラフ ネットワークでこのスコアを上回ることができるかどうか見てみましょう。

In [18]:
node_gnn_model, node_gnn_result = train_node_classifier(model_name="GNN",
                                                        layer_name="GCN",
                                                        dataset=cora_dataset, 
                                                        c_hidden=16, 
                                                        num_layers=2,
                                                        dp_rate=0.1)
print_results(node_gnn_result)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.2 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../saved_models/tutorial7/NodeLevelGNN.ckpt`
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...
Train accuracy: 100.00%
Val accuracy:   78.00%
Test accuracy:  82.40%


出来レースですが、GNN モデルは MLP をかなり上回っています。これは、グラフ情報を使用すると実際に予測が改善され、より一般化できることを示しています。

モデルのハイパーパラメーターは、比較的小さなネットワークを作成するために選択されています。これは、入力次元が 1433 の最初のレイヤーは、大規模なグラフに対して実行するのに比較的コストがかかる可能性があるためです。一般に、GNN は非常に大きなグラフでは比較的高価になる可能性があります。これが、そのような GNN が小さな隠れサイズを持っているか、大きな元のグラフの接続されたサブグラフをサンプリングする特別なバッチ戦略を使用する理由です。

### Section3-2. エッジレベルのタスク: リンク予測

一部のアプリケーションでは、ノード レベルではなくエッジ レベルで予測する必要がある場合があります。 GNN で最も一般的なエッジレベルのタスクはリンク予測です。リンク予測とは、与えられたグラフで、2 つのノード間にエッジがあるかどうか、またはあるべきかどうかを予測することを意味します。たとえば、ソーシャル ネットワークでは、これは Facebook などによって新しい友達を提案するために使用されます。繰り返しますが、グラフ レベルの情報は、このタスクを実行するために非常に重要です。出力予測は、通常、ノード フィーチャのペアに対して類似性メトリックを実行することによって行われます。リンクがある場合は 1、そうでない場合は 0 に近くなります。勉強会自体の時間が限られているため、このタスクは本勉強会では実装しません。とはいえ、このタスクを詳しく調べたい場合は、優れたリソースがたくさんあります。
今回の勉強会の主軸となる論文は以下のとおりです。

* [PyTorch Geometric example](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/link_pred.py)
* [Graph Neural Networks: A Review of Methods and Applications](https://arxiv.org/pdf/1812.08434.pdf), Zhou et al. 2019
* [Link Prediction Based on Graph Neural Networks](https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf), Zhang and Chen, 2018.

* [深層学習を用いた未知ノードの出現を考慮した時系列グラフの予測](https://db-event.jpn.org/deim2019/post/papers/197.pdf), 山崎 翔平, 原田 圭, 佐々木勇和, 鬼塚 真

* 

### Section3-3. グラフレベルのタスク: グラフの分類

最後に、本勉強会のこの部分では、GNN をグラフ分類のタスクに適用する方法を詳しく見ていきます。目標は、単一のノードまたはエッジではなく、グラフ全体を分類することです。したがって、いくつかの構造グラフ プロパティに基づいて分類する必要がある複数のグラフのデータセットも与えられます。グラフ分類の最も一般的なタスクは、分子をグラフとして表現する分子特性予測です。各原子はノードにリンクされており、グラフのエッジは原子間の結合です。たとえば、下の図を見てください。

![任意の画像名を付ける](https://drive.google.com/uc?id=1814uM_tXHYqYOVmbcCmqno6BOirOpMP8)

左側には、異なる原子を持つ任意の小さな分子があり、画像の右側にはグラフ表現が示されています。原子タイプはノード機能 (ワンホット ベクトルなど) として抽象化され、さまざまな結合タイプがエッジ機能として使用されます。簡単にするために、このチュートリアルではエッジ属性を無視しますが、[リレーショナル グラフ畳み込み](https://arxiv.org/abs/1703.06103) のような方法を使用して含めることができます。これは、エッジ タイプごとに異なる重み行列を使用します。 

以下で使用するデータセットは、MUTAG データセットと呼ばれます。これは、グラフ分類アルゴリズムの一般的な小さなベンチマークであり、グラフごとに平均 18 個のノードと 20 個のエッジを持つ 188 個のグラフが含まれています。グラフ ノードには 7 つの異なるラベル/原子タイプがあり、バイナリ グラフ ラベルは「特定のグラム陰性菌に対する突然変異誘発効果」を表します (ラベルの特定の意味はここではあまり重要ではありません)。データセットは、[TUDatasets](https://chrsmrrs.github.io/datasets/) として知られるさまざまなグラフ分類データセットの大規模なコレクションの一部であり、`torch_geometric.datasets.TUDataset` ([documentation ](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset)) PyTorch ジオメトリックで。以下のデータセットをロードできます。

In [19]:
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="MUTAG")

Extracting ../data/MUTAG/MUTAG.zip
Processing...
Done!


データセットの統計をいくつか見てみましょう。

In [20]:
print("Data object:", tu_dataset.data)
print("Length:", len(tu_dataset))
print(f"Average label: {tu_dataset.data.y.float().mean().item():4.2f}")

Data object: Data(x=[3371, 7], edge_index=[2, 7442], edge_attr=[7442, 4], y=[188])
Length: 188
Average label: 0.66


/usr/local/lib/python3.9/dist-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


最初の行は、データセットがさまざまなグラフを格納する方法を示しています。各グラフのノード、エッジ、およびラベルは 1 つのテンソルに連結され、データセットはそれに応じてテンソルを分割するインデックスを格納します。データセットの長さはグラフの数であり、「平均ラベル」はラベル 1 のグラフのパーセンテージを示します。パーセンテージが 0.5 の範囲内にある限り、比較的バランスの取れたデータセットがあります。グラフ データセットが非常に不均衡であることはよくあることです。そのため、クラスのバランスをチェックすることは常に良いことです。

次に、データセットをトレーニングとテストの部分に分割します。データセットのサイズが小さいため、今回は検証セットを使用しないことに注意してください。したがって、評価のノイズにより、モデルが検証セットにわずかにオーバーフィットする可能性がありますが、トレーニングされていないデータでのパフォーマンスの推定値はまだ得られます。

In [21]:
torch.manual_seed(42)
tu_dataset.shuffle()
train_dataset = tu_dataset[:150]
test_dataset = tu_dataset[150:]

データ ローダーを使用すると、$N$ グラフのバッチ処理で問題が発生します。バッチ内の各グラフは異なる数のノードとエッジを持つことができるため、単一のテンソルを取得するには多くのパディングが必要になります。 Torch ジオメトリックは、別のより効率的なアプローチを使用します: $N$ グラフを、ノードとエッジ リストが連結された単一の大きなグラフとしてバッチで表示できます。 $N$ グラフ間にエッジがないため、大きなグラフで GNN レイヤーを実行すると、各グラフで GNN を個別に実行した場合と同じ出力が得られます。視覚的に、このバッチ処理戦略は以下に視覚化されています (図のクレジット - PyTorch 幾何学チーム、[チュートリアルはこちら](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=2owRWKcuoALo))。

![任意の画像名を付ける](https://drive.google.com/uc?id=1HLivk_GuH446bMjnHYk82YGc3kbUSubO)

隣接行列は、2 つの異なるグラフに由来するノードの場合はゼロであり、それ以外の場合は、個々のグラフの隣接行列に従います。幸いなことに、この戦略はトーチ ジオメトリックで既に実装されているため、対応するデータ ローダーを使用できます。

In [22]:
graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=64) # Additional loader if you want to change to a larger dataset
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

以下のバッチをロードして、バッチ処理の動作を確認しましょう。

In [23]:
batch = next(iter(graph_test_loader))
print("Batch:", batch)
print("Labels:", batch.y[:10])
print("Batch indices:", batch.batch[:40])

Batch: DataBatch(edge_index=[2, 1512], x=[687, 7], edge_attr=[1512, 4], y=[38], batch=[687], ptr=[39])
Labels: tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0])
Batch indices: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])


テスト データセット用に 38 個のグラフが積み上げられています。 `batch` に格納されているバッチ インデックスは、最初の 12 個のノードが最初のグラフに属し、次の 22 個が 2 番目のグラフに属することを示しています。これらのインデックスは、最終的な予測を実行するために重要です。グラフ全体で予測を実行するには、通常、GNN モデルを実行した後、すべてのノードでプーリング操作を実行します。この場合、平均プーリングを使用します。したがって、どの平均プールにどのノードを含める必要があるかを知る必要があります。このプーリングを使用して、以下のグラフ ネットワークを作成できます。具体的には、以前のクラス「GNNModel」を再利用し、グラフ予測タスク用に平均プールと単一の線形レイヤーを追加するだけです。

In [24]:
class GraphGNNModel(nn.Module):
    
    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        """
        Inputs:
            c_in - Dimension of input features
            c_hidden - Dimension of hidden features
            c_out - Dimension of output features (usually number of classes)
            dp_rate_linear - Dropout rate before the linear layer (usually much higher than inside the GNN)
            kwargs - Additional arguments for the GNNModel object
        """
        super().__init__()
        self.GNN = GNNModel(c_in=c_in, 
                            c_hidden=c_hidden, 
                            c_out=c_hidden, # Not our prediction output yet!
                            **kwargs)
        self.head = nn.Sequential(
            nn.Dropout(dp_rate_linear),
            nn.Linear(c_hidden, c_out)
        )

    def forward(self, x, edge_index, batch_idx):
        """
        Inputs:
            x - Input features per node
            edge_index - List of vertex index pairs representing the edges in the graph (PyTorch geometric notation)
            batch_idx - Index of batch element for each node
        """
        x = self.GNN(x, edge_index)
        x = geom_nn.global_mean_pool(x, batch_idx) # Average pooling
        x = self.head(x)
        return x

最後に、トレーニングを処理する PyTorch Lightning モジュールを作成できます。これは、以前に見たモジュールに似ており、トレーニングに関して驚くべきことは何もしません。バイナリ分類タスクがあるため、バイナリ クロス エントロピー損失を使用します。

In [25]:
class GraphLevelGNN(pl.LightningModule):
    
    def __init__(self, **model_kwargs):
        super().__init__()
        # Saving hyperparameters
        self.save_hyperparameters()
        
        self.model = GraphGNNModel(**model_kwargs)
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch
        x = self.model(x, edge_index, batch_idx)
        x = x.squeeze(dim=-1)
        
        if self.hparams.c_out == 1:
            preds = (x > 0).float()
            data.y = data.y.float()
        else:
            preds = x.argmax(dim=-1)
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=1e-2, weight_decay=0.0) # High lr because of small dataset and small model
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

以下では、データセットでモデルをトレーニングします。これまで見てきた典型的なトレーニング関数に似ています。

In [26]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)
    
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=500,
                         enable_progress_bar=False)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"GraphLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)
        model = GraphLevelGNN(c_in=tu_dataset.num_node_features, 
                              c_out=1 if tu_dataset.num_classes==2 else tu_dataset.num_classes, 
                              **model_kwargs)
        trainer.fit(model, graph_train_loader, graph_val_loader)
        model = GraphLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    # Test best model on validation and test set
    train_result = trainer.test(model, graph_train_loader, verbose=False)
    test_result = trainer.test(model, graph_test_loader, verbose=False)
    result = {"test": test_result[0]['test_acc'], "train": train_result[0]['test_acc']} 
    return model, result

最後に、トレーニングとテストを実行しましょう。さまざまな GNN レイヤー、ハイパーパラメーターなどを自由に試してみてください。

In [27]:
model, result = train_graph_classifier(model_name="GraphConv", 
                                       c_hidden=256, 
                                       layer_name="GraphConv", 
                                       num_layers=3, 
                                       dp_rate_linear=0.5,
                                       dp_rate=0.0)

INFO:lightning_fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.0.2 to v2.0.1. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../saved_models/tutorial7/GraphLevelGraphConv.ckpt`
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...


/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(
/usr/local/lib/python3.9/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [28]:
print(f"Train performance: {100.0*result['train']:4.2f}%")
print(f"Test performance:  {100.0*result['test']:4.2f}%")

Train performance: 93.28%
Test performance:  92.11%


テストのパフォーマンスは、データセットの目に見えない部分で非常に良いスコアが得られることを示しています。テストセットを検証にも使用しているため、このセットにわずかにオーバーフィットしている可能性があることに注意してください。それにもかかわらず、実験は、GNNがグラフや分子の特性を予測するのに非常に強力である可能性があることを示しています.

## Section4. 結論

この勉強会では、グラフ構造へのニューラル ネットワークの適用を見てきました。グラフを表現する方法 (隣接行列またはエッジ リスト) を調べ、一般的なグラフ レイヤーである GCN と GAT の実装について説明しました。実装は、多くの場合、理論よりも簡単なレイヤーの実用的な側面を示しました。最後に、ノード、エッジ、グラフ レベルでさまざまなタスクを試しました。全体として、予測にグラフ情報を含めることが、高いパフォーマンスを達成するために重要であることがわかりました。 GNN の恩恵を受けるアプリケーションは数多くあり、これらのネットワークの重要性は今後数年間で高まる可能性があります。